In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
TARGET_COL = "CCS" # Ou "Log_CCS" após transformação
PREDICTOR_COLS_COMPOSITION = ["Gordura", "Proteina", "Lactose", "Densidade"]
CATEGORICAL_COL = "Estação"

PREPROCESSED_FILE =  '/content/drive/MyDrive/TCC/dados_leite_preprocessados.csv'


In [ ]:
dados = pd.read_csv(PREPROCESSED_FILE)
print(dados.shape)


(184993, 27)


In [ ]:
dados[PREDICTOR_COLS_COMPOSITION + [TARGET_COL]] = dados[PREDICTOR_COLS_COMPOSITION + [TARGET_COL]].apply(pd.to_numeric, errors="coerce")
print(dados.shape)

(184993, 27)


# Regressão linear

In [ ]:
#var dummy para Estação (para Regressão Linear)
if dados[CATEGORICAL_COL].nunique() > 1:
    print(f'Criando variáveis dummy para "{CATEGORICAL_COL}"...')
    dados = pd.get_dummies(dados, columns=[CATEGORICAL_COL], drop_first=True, prefix=CATEGORICAL_COL)
    dummy_cols = [col for col in dados.columns if col.startswith(f"{CATEGORICAL_COL}_")]
    print(f"Colunas dummy criadas: {dummy_cols}")
    PREDICTORS_ALL = PREDICTOR_COLS_COMPOSITION + dummy_cols
else:
    print(f'Coluna "{CATEGORICAL_COL}" não possui categorias suficientes para dummies ou não existe.')
    dummy_cols = []
    PREDICTORS_ALL = PREDICTOR_COLS_COMPOSITION

print(f"Preditoras finais para modelos: {PREDICTORS_ALL}")


Criando variáveis dummy para "Estação"...
Colunas dummy criadas: ['Estação_Outono', 'Estação_Primavera', 'Estação_Verão']
Preditoras finais para modelos: ['Gordura', 'Proteina', 'Lactose', 'Densidade', 'Estação_Outono', 'Estação_Primavera', 'Estação_Verão']


In [ ]:
print("\n--- Regressão Linear Simples (Lactose vs CCS) ---")
X_simple = dados["Lactose"]
y_simple = dados[TARGET_COL]
X_simple = sm.add_constant(X_simple) # Adiciona intercep

model_simple = sm.OLS(y_simple, X_simple).fit()

print(model_simple.summary())



--- Regressão Linear Simples (Lactose vs CCS) ---
                            OLS Regression Results                            
Dep. Variable:                    CCS   R-squared:                       0.140
Model:                            OLS   Adj. R-squared:                  0.140
Method:                 Least Squares   F-statistic:                 3.017e+04
Date:                Wed, 04 Jun 2025   Prob (F-statistic):               0.00
Time:                        13:49:19   Log-Likelihood:            -2.5716e+06
No. Observations:              184993   AIC:                         5.143e+06
Df Residuals:                  184991   BIC:                         5.143e+06
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
c

In [ ]:
print(f"\n---  Regressão Linear Múltipla (Predizendo {TARGET_COL}) ---")

#esse codigo abaixo foi gerado pelo gemini, pois o meu estava dando MissingDataError: exog contains inf or nans

# Create a list of columns to check for NaNs
cols_to_check_for_nan = PREDICTORS_ALL + [TARGET_COL]

# Drop rows where any of the specified columns have NaN values
print(f"Shape dos dados antes de remover NaNs: {dados.shape}")
dados_cleaned = dados.dropna(subset=cols_to_check_for_nan)
print(f"Shape dos dados depois de remover NaNs nos preditores/target: {dados_cleaned.shape}")

# Use the cleaned data for modeling
X_multi = dados_cleaned[PREDICTORS_ALL].astype(float)
y_multi = dados_cleaned[TARGET_COL].astype(float)
X_multi = sm.add_constant(X_multi) # Adiciona intercepto

model_multi = sm.OLS(y_multi, X_multi).fit()
print(model_multi.summary())


---  Regressão Linear Múltipla (Predizendo CCS) ---
Shape dos dados antes de remover NaNs: (184993, 29)
Shape dos dados depois de remover NaNs nos preditores/target: (167240, 29)
                            OLS Regression Results                            
Dep. Variable:                    CCS   R-squared:                       0.151
Model:                            OLS   Adj. R-squared:                  0.151
Method:                 Least Squares   F-statistic:                     4249.
Date:                Wed, 04 Jun 2025   Prob (F-statistic):               0.00
Time:                        13:49:19   Log-Likelihood:            -2.3233e+06
No. Observations:              167240   AIC:                         4.647e+06
Df Residuals:                  167232   BIC:                         4.647e+06
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                        coef  

In [ ]:


# --- Modelo 3: Random Forest Regressor (Predizendo CCS) ---

print(f"\n--- Modelo 3: Random Forest Regressor (Predizendo {TARGET_COL}) ---")

X_rf = dados[PREDICTORS_ALL].astype(float)
y_rf = dados[TARGET_COL].astype(float)

X_train, X_test, y_train, y_test = train_test_split(X_rf, y_rf, test_size=0.3, random_state=42)

# Treinar
rf_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1, max_depth=10, min_samples_leaf=5)
print("Treinando...")
rf_model.fit(X_train, y_train)


y_pred_rf = rf_model.predict(X_test)

# Avaliar
mse_rf = mean_squared_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)
print(f" (MSE): {mse_rf:.4f}")
print(f"(R²): {r2_rf:.4f}")

importances = rf_model.feature_importances_
feature_importance_df = pd.DataFrame({"Feature": PREDICTORS_ALL, "Importance": importances}).sort_values(by="Importance", ascending=False)
print(feature_importance_df)



--- Modelo 3: Random Forest Regressor (Predizendo CCS) ---
Treinando...
 (MSE): 66375602637.7042
(R²): 0.1752
             Feature  Importance
2            Lactose    0.724306
0            Gordura    0.112175
1           Proteina    0.099240
3          Densidade    0.022359
4     Estação_Outono    0.021800
6      Estação_Verão    0.010495
5  Estação_Primavera    0.009626
